# Część badawcza
Skoroszyt przedstawia część badawczą pracy magisterskiej. Składa się z analizy danych z aplikacji Zapunktuj oraz analizy wyników ankiety przeprowadzonej wśród studentów.


## Wstęp

W tym rozdziale zostały przedstawione wyniki przeprowadzonych badań oraz proces samej analizy w środowisku Jupyter z wykorzystaniem wcześniej opisanych narzędzi. Przeanalizowano wyniki oceniania z autorskiej aplikacji Zapunktuj oraz wyniki ankiety przeprowadzonej wśród studentów.

Podczas analizy starano się odpowiedzieć na następujące pytania:

* Czy kolejność wyświetlania prac do oceny miała wpływ na liczbę zebranych punktów przez sekcję projektową?
* Czy niezależnie od systemu oceniania w aplikacji Zapunktuj ranking najlepszych prac prezentował te same projekty?
* Jaką liczbę prac powinna oceniać jedna osoba?
* Czy najlepiej oceniane projekty w aplikacji Zapunktuj otrzymały najwięcej dodatkowych wyróżnień?
* Jak studenci postrzegają rezultaty oceny wzajemnej, zarówno oceny, jak i otrzymane informacje zwrotne?
* Jaką funkcję powinna pełnić ocena koleżeńska?
* Jak studenci postrzegają ocenę od nauczyciela w stosunku do oceny rówieśniczej?
* Czy studenci wolą pozostać anonimowi podczas oceny?
* Jak świadomość zobaczenia pracy przez rówieśników wpływa na motywację do wykonania pracy?
* Jakie trudności występowały podczas oceny wzajemnej oraz, które kwestie były najbardziej problematyczne?
* Jakie znaczenie miały kwestie interpersonalne podczas przeprowadzania oceny?
* Jak studenci oceniają użyte przez nich aplikacje do przeprowadzenia oceny wzajemnej?
* Które sposoby oceny wzajemnej i jakie ich elementy najbardziej przypadły do gustu studentom?
* Jak studenci oceniają przydatność konkretnych kryteriów oceny przygotowanych przez nauczyciela?
* Co można poprawić w aplikacji Zapunktuj?


Zbiory danych zostały odpowiednio przygotowane przed rozpoczęciem analizy ich wyników, a cały proces został opisany poniżej.

## Przygotowanie

W pierwszej kolejności zaimportowano  wszystkie potrzebne biblioteki oraz moduły, potrzebne do wykonania analizy.

In [1]:
# import bibliotek
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from IPython.display import display
from google.colab import data_table
from string import digits


# konfiguracja
data_table.enable_dataframe_formatter() # better looking tables
pd.options.mode.chained_assignment = None  # default='warn' - for warnings
config = {'toImageButtonOptions': { 'format':'png', 'filename':'plot', 'scale':2}}

# kolory
color_discrete_sequence =  ['#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377', '#BBBBBB'] # bright - default order
color_discrete_sequence =  ['#4477AA', '#EE6677', '#228833', '#CCBB44', '#AA3377', '#66CCEE', '#BBBBBB'] # bright
color_continous_sequence = [ '#FFFFE5', '#FFF7BC', '#FEE391', '#FEC44F', '#FB9A29', '#EC7014', '#CC4C02', '#993404', '#662506']
pio.templates["my_template"] = go.layout.Template(layout=go.Layout(colorway=color_discrete_sequence, colorscale=dict(sequential=color_continous_sequence)))
pio.templates.default = 'plotly+my_template'

# pio.renderers.default = 'notebook'
# pio.kaleido.scope.default_format = "png"

Plik w formacie .xlsx z odpowiedziami do ankiety został pobrany, a następnie umieszczony na wirtualnym dysku. Pliki .csv z wynikami oceniania z aplikacji Zapunktuj zamieszczone zostały w tym samym miejscu. Następnie pliki zostały odczytane.

In [2]:
# czytanie plików z dysku wirtualnego
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# plik z wynikami ankiety
form_path = '/content/drive/MyDrive/Dyplom/MS_summer_20-21.xlsx'
df_form = pd.read_excel(form_path, 'Form1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# wyniki z aplikacji Zapuntuj

# pominięte zostają wiersze 0, 2, 3 gdyż nie zawierają potrzebnych informacji
results_path = '/content/drive/MyDrive/Dyplom/Wyniki-Zapunktuj/'
results_1_file = '1_MS-PGK_z19-20-wyniki.csv'
df_results_1 = pd.read_csv(results_path + results_1_file, skiprows = [0, 2, 3])
# df_results_1 = pd.read_csv('/content/drive/MyDrive/Dyplom/Wyniki-Zapunktuj/1_MS-PGK_z19-20-wyniki.csv', skiprows = [0, 2, 3])
df_results_2 = pd.read_csv('/content/drive/MyDrive/Dyplom/Wyniki-Zapunktuj/2_RP N -RP,_niest,_l_19-20-wyniki.csv', skiprows = [0, 2, 3])
df_results_3 = pd.read_csv('/content/drive/MyDrive/Dyplom/Wyniki-Zapunktuj/3_MS,s-MS_stacj_l,19-20-wyniki.csv', skiprows = [0, 2, 3])
df_results_4 = pd.read_csv('/content/drive/MyDrive/Dyplom/Wyniki-Zapunktuj/4_R2021z-Rynek_pracy_s,z20-21-wyniki.csv', skiprows = [0, 2, 3])
df_results_5 = pd.read_csv('/content/drive/MyDrive/Dyplom/Wyniki-Zapunktuj/5_MS2021-MS_pgk_z_20-21-wyniki.csv', skiprows = [0, 2, 3])
df_results_6 = pd.read_csv('/content/drive/MyDrive/Dyplom/Wyniki-Zapunktuj/6_MS2021-MS,_s,_lato_20_21-wyniki.csv', skiprows = [0, 2, 3])

In [4]:
# Wyniki Zapunktuj - revert changes to initial dataset (without loading file from drive)
# df_results_1 = results_file_1.copy()

In [5]:
# Ankieta - revert changes to initial dataset (without loading file from drive)
# df_form = form.copy()

## Analiza danych z aplikacji Zapunktuj



Na początku sprawdzony zostaje rozmiar danych oraz atrybuty zbioru. Zbiór składa się z 23 wierszy i 9 kolumn.

In [6]:
# rozmiar danych
print(df_results_1.shape, df_results_1.columns, sep='\n\n')

(23, 9)

Index(['Sekcja', 'Tytuł projektu', 'Punkty z oceniania sposobem 1',
       'Punkty z oceniania sposobem 2', 'Punkty z oceniania sposobem 3',
       'Punkty od nauczyciela', 'Punkty od nauczyciela (poprawa)',
       'Wyróżnienia (studenci)', 'Wyróżnienie (nauczyciel)'],
      dtype='object')


In [7]:
# atrybuty zbioru
df_results_1.columns

Index(['Sekcja', 'Tytuł projektu', 'Punkty z oceniania sposobem 1',
       'Punkty z oceniania sposobem 2', 'Punkty z oceniania sposobem 3',
       'Punkty od nauczyciela', 'Punkty od nauczyciela (poprawa)',
       'Wyróżnienia (studenci)', 'Wyróżnienie (nauczyciel)'],
      dtype='object')

Każdy rekord przedstawia nazwę sekcji projektowej, tytuł projektu oraz liczbę punktów i wyróżnień uzyskanych od innych osób w procesie oceniania różnymi metodami. Kolejno sprawdzono pierwsze i ostatnie rekordy zbioru.

In [8]:
df_results_1.head(3)

,Sekcja,Tytuł projektu,Punkty z oceniania sposobem 1,Punkty z oceniania sposobem 2,Punkty z oceniania sposobem 3,Punkty od nauczyciela,Punkty od nauczyciela (poprawa),Wyróżnienia (studenci),Wyróżnienie (nauczyciel)
0,Sekcja 1,Król Kokainy w liczbach,41,0,0.0/0,0/0,0/0,0,nie
1,Sekcja 2,Jak dbać o sen?,8,0,0.0/0,0/0,0/0,0,nie
2,Sekcja 3,Moduł społeczny - Infografika,16,0,0.0/0,0/0,0/0,0,nie


In [9]:
df_results_6.tail(5)

,Sekcja,Tytuł projektu,Punkty z oceniania sposobem 1,Punkty z oceniania sposobem 2,Punkty z oceniania sposobem 3,Punkty od nauczyciela,Punkty od nauczyciela (poprawa),Wyróżnienia (studenci),Wyróżnienie (nauczyciel)
38,Sekcja 39,Kobiety w IT,17.0,12.0,2.14/3,0/3,0/3,0.0,nie
39,Sekcja 40,101 modelu pamięci Rust,11.0,14.0,1.76/3,0/3,0/3,0.0,nie
40,Oceniający sposobem 1:,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Oceniający sposobem 2:,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,Oceniający sposobem 3:,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Patrząc na pierwsze i ostatnie rekordy, zauważono, że ostatnie 3 nie stanowią danych odpowiadających zdefiniowanym w nagłówku kolumnom.
Są to liczby odpowiadające liczbie osób, jaka brała udział w ocenianiu poszczególnymi sposobami.
<!-- Można te wartości zapisać do osobnych zmiennych, a następnie usunąć rekordy w których występują. -->
Informacje te nie były potrzebne w analizie, więc rekordy w których występują, zostały usunięte.
<!-- W pierwszych grupach studenckich oceniano tylko metodą pierwszą, więc potrzebna jest w ich wypadku tylko jedna wartość do zapisania przed usunięciem.  -->
Atrybut sekcja zawierający nazwę sekcji został ustawiony natomiast jako index zbioru, gdyż każdy rekord zawiera unikalną wartość (nazwę sekcji projektowej generowanej przez system).

In [10]:
df_results_1.set_index('Sekcja', inplace=True)
df_results_1.drop(df_results_1.tail(3).index, inplace=True)

Zauważono też, że część rekordów jest pusta ze względu na to, że w grupie studenckiej było mniej sekcji projektowych niż sekcji przygotowanych w aplikacji na etapie konfiguracji. Niektóre z kolumn nie zawierały żadnych informacji. Są to między innymi kolumny "Punkty od nauczyciela", "Punkty od nauczyciela (poprawa)" oraz "Wyróżnienie (nauczyciel)". Pozostały one puste, ponieważ w badanych grupach nauczyciel nie oceniał projektów w aplikacji.  
Te kolumny oraz puste rekordy należało usunąć. Pozostałe puste kolumny dla niektórych zbiorów zawierały informacje, dlatego nie zostają usunięte, aby każdy z badanych sześciu zbiorów miał jednakową ilość atrybutów. Są to kolumny "Punkty z oceniania sposobem 2", "Punkty z oceniania sposobem 3" oraz "Wyróżnienia (studenci)".

In [11]:
# usunięcie pustych sekcji
df_results_1 = df_results_1[df_results_1.iloc[:,0] != '---']
df_results_1.drop(['Punkty od nauczyciela', 'Punkty od nauczyciela (poprawa)', 'Wyróżnienie (nauczyciel)'], axis=1, inplace=True)

W zbiorze sprawdzone zostały także typy danych.

In [12]:
df_results_1.dtypes

Tytuł projektu                   object
Punkty z oceniania sposobem 1    object
Punkty z oceniania sposobem 2    object
Punkty z oceniania sposobem 3    object
Wyróżnienia (studenci)           object
dtype: object

Powyżej widać, że wszystkie typy atrybutów zbioru są typami ogólnymi. Typ danych atrybutu zawierającego sumę punktów przyznaną na projekt został zamieniony na typ całkowitoliczbowy.

In [13]:
df_results_1 = df_results_1.astype({'Punkty z oceniania sposobem 1': 'int', 'Punkty z oceniania sposobem 2': 'int', 'Wyróżnienia (studenci)': 'int'})

Wszystkie poprzednie operacje zostały wykonane dla pozostałych 5 zbiorów zawierających dane z wynikami oceniania w innych grupach.

In [14]:
def clean_results(data):
  data.set_index('Sekcja', inplace=True)
  data.drop(data.tail(3).index, inplace=True)
  data = data[data.iloc[:,0] != '---']
  data = data.astype({'Punkty z oceniania sposobem 1': 'int', 'Punkty z oceniania sposobem 2': 'int', 'Wyróżnienia (studenci)': 'int'})
  data.drop(['Punkty od nauczyciela', 'Punkty od nauczyciela (poprawa)', 'Wyróżnienie (nauczyciel)'], axis=1, inplace=True)
  return data

df_results_2 = clean_results(df_results_2)
df_results_3 = clean_results(df_results_3)
df_results_4 = clean_results(df_results_4)
df_results_5 = clean_results(df_results_5)
df_results_6 = clean_results(df_results_6)

Aby zweryfikować, czy kolejność wyświetlania prac do oceny mogła mieć wpływ na końcowe wyniki, zbiór został podzielony na 2 równe części. Następnie dla każdej z nich punkty zostały zsumowane i przedstawione na wykresie.

In [15]:
# plot 1 - podział zbioru na 2 równe części
df_split = np.array_split(df_results_1, 2)

sums_df = pd.DataFrame({'połowa': ['pierwsza połowa prac', 'druga połowa prac'],
                        'Punkty z oceniania sposobem 1': [df_split[0]['Punkty z oceniania sposobem 1'].sum(),
                                                          df_split[1]['Punkty z oceniania sposobem 1'].sum()]})

percent_change = 100*(sums_df.iloc[1,1] - sums_df.iloc[0,1]) / sums_df.iloc[0,1]

fig = px.bar(sums_df, x='połowa', y='Punkty z oceniania sposobem 1',
             color="połowa",
             text_auto=True, width=800,
             color_discrete_sequence=color_discrete_sequence
             )

fig.add_annotation(x=1, y=sums_df.iloc[1,1], text=f"{percent_change:.2f}%", showarrow=False, yshift=20, font=dict(color="red",size=14),)
fig.update_layout(showlegend=False, width=500, height=400, margin=dict(l=20, r=20, t=10, b=10), xaxis_title=None)

fig.show(config=config)

Na powyższym wykresie widać, że pierwsza połowa prac otrzymała zdecydowanie więcej punktów. Druga połowa otrzymała ich blisko 43% mniej. Analizowany zbiór można podzielić także na 4 części i dokonać dla niego podobnej wizualizacji.

In [16]:
# plot 2 -
df_split = np.array_split(df_results_1, 4)

sums_df = pd.DataFrame({'ćwiartka': ['pierwsza', 'druga', 'trzecia', 'czwarta'], 'Punkty z oceniania sposobem 1': [df_split[0]['Punkty z oceniania sposobem 1'].sum(),
                                                                                                                   df_split[1]['Punkty z oceniania sposobem 1'].sum(),
                                                                                                                   df_split[2]['Punkty z oceniania sposobem 1'].sum(),
                                                                                                                   df_split[3]['Punkty z oceniania sposobem 1'].sum()
                                                                                                                   ]})
fig = px.bar(sums_df, x='ćwiartka', y='Punkty z oceniania sposobem 1',
             color="ćwiartka",
             text_auto=True, width=800)
fig.update_layout(showlegend=False, width=500, height=300, margin=dict(l=20, r=20, t=10, b=10))

fig.show(config=config)

Po podziale na 4 części nie widać już tak wyraźnej zależności. Poniżej porównane zostały także wyniki dla drugiej grupy.

In [17]:
# plot 3
df_split_2 = np.array_split(df_results_2, 2)

sums_df_2 = pd.DataFrame({'połowa': ['pierwsza połowa prac', 'druga połowa prac'], 'Punkty z oceniania sposobem 1': [df_split_2[0]['Punkty z oceniania sposobem 1'].sum(), df_split_2[1]['Punkty z oceniania sposobem 1'].sum()]})

percent_change_2 = 100*(sums_df_2.iloc[1,1] - sums_df_2.iloc[0,1]) / sums_df_2.iloc[0,1]

fig_2 = px.bar(sums_df_2, x='połowa', y='Punkty z oceniania sposobem 1',
             text_auto=True, width=800, color="połowa"
             )

fig_2.add_annotation(x=1, y=sums_df_2.iloc[1,1], text=f"{percent_change_2:.2f}%", showarrow=False, yshift=20, font=dict(color="red",size=14),)
fig_2.update_layout(showlegend=False, width=500, height=400, margin=dict(l=20, r=20, t=10, b=10), xaxis_title=None)
fig_2.show(config=config)

Dla drugiej grupy wyniki prezentują podobną zależność. W tym wypadku suma punktów dla drugiej części projektów była o blisko 50% mniejsza.

In [18]:
# plot 4
df_split_2 = np.array_split(df_results_2, 4)
sums_df_2 = pd.DataFrame({'ćwiartka': ['pierwsza', 'druga', 'trzecia', 'czwarta'],
                          'Punkty z oceniania sposobem 1': [df_split_2[0]['Punkty z oceniania sposobem 1'].sum(),
                                                            df_split_2[1]['Punkty z oceniania sposobem 1'].sum(),
                                                            df_split_2[2]['Punkty z oceniania sposobem 1'].sum(),
                                                            df_split_2[3]['Punkty z oceniania sposobem 1'].sum()
                                                            ]})

fig = px.bar(sums_df_2, x='ćwiartka', y='Punkty z oceniania sposobem 1',
             title='Suma punktów dla czterech ćwiartek - Grupa 2', color="ćwiartka",
             text_auto=True, width=800,
             )
fig.show(config=config)

Po przedstawieniu danych pokazujących sumy punktów w czterech ćwiartkach dla drugiej grupy, widać nadal wyraźnie większą liczbę punktów dla pierwszej połowy projektów. Nie widać natomiast wyraźnego spadku pomiędzy każdą z ćwiartek.
Poniżej przedstawione zostały wykresy dla pozostałych 4 grup, gdzie również zbiory podzielone zostały na dwie części.

In [19]:
# plot 5
sums_list = list()

for i, results in enumerate((df_results_3, df_results_4, df_results_5, df_results_6)):
  df_split = np.array_split(results, 2)
  sums = pd.DataFrame({'połowa': ['pierwsza<br>połowa', 'druga<br>połowa'], 'Punkty z oceniania sposobem 1': [df_split[0]['Punkty z oceniania sposobem 1'].sum(), df_split[1]['Punkty z oceniania sposobem 1'].sum()]})
  sums["Grupa"] = i+3
  sums_list.append(sums)

df_concat = pd.concat(sums_list)

fig = px.bar(df_concat, x="połowa", y="Punkty z oceniania sposobem 1", facet_col="Grupa", color="połowa", text_auto=True, width=1000)

for i in range(df_concat['Grupa'].nunique()):
  pos = i * 2 + 1
  percent_change = 100*(df_concat.iloc[pos,1] - df_concat.iloc[pos-1,1]) / df_concat.iloc[pos-1,1]
  fig.add_annotation(x=1, y=df_concat.iloc[pos,1], text=f"{percent_change:.2f}%", showarrow=False, yshift=30, font=dict(color="red",size=14), xref=f'x{i+1}')

fig.update_xaxes(title_text=None)
fig.update_layout(showlegend=False, width=600, height=400, margin=dict(l=20, r=20, t=30, b=10))
fig.show(config=config)

Po sprawdzeniu danych dla każdej z 6 grup okazuje się, że w każdej z nich występuje to samo zjawisko.
Podział zbiorów sekcji na dwie równe części i zsumowanie punktów w nich uzyskanych pokazuje, że za każdym razem druga połowa sekcji uzyskiwała mniej punktów niż pierwsza połowa - od niecałych 5% do blisko 50%.
Aby dokładniej zbadać to zjawisko, postanowiono zaimplementować w Aplikacji zapunktuj drugi system oceniania, który od pierwszego różni się tym, że kolejność prac wyświetlanych podczas oceny jest pseudolosowa dla każdej z oceniających osób. Porównanie wyników oceny pierwszym i drugim sposobem w danej grupie pozwoliły sprawdzić, czy wyniki różniły się znacząco, jeśli prace były pomieszane.

In [20]:
print('Grupy oceniające metodą drugą z mieszaniem kolejności prac:')
for i, data in enumerate(tuple([df_results_1, df_results_2, df_results_3, df_results_4, df_results_5, df_results_6])):
  print(f"Grupa {i+1} - {'tak' if data['Punkty z oceniania sposobem 2'].sum() > 0 else 'nie'}")

Grupy oceniające metodą drugą z mieszaniem kolejności prac:
Grupa 1 - nie
Grupa 2 - nie
Grupa 3 - nie
Grupa 4 - tak
Grupa 5 - tak
Grupa 6 - tak


Pierwsze 3 grupy oceniały tylko sposobem pierwszym, więc dane zostanę zaprezentowano tylko dla grup 4, 5 oraz 6.

In [21]:
# plot 6
sums_list = list()

for i, results in enumerate((df_results_4, df_results_5, df_results_6)):
  df_split = np.array_split(results, 2)
  sums = pd.DataFrame({'połowa': ['pierwsza<br>połowa', 'druga<br>połowa'], 'Punkty z oceniania sposobem 2': [df_split[0]['Punkty z oceniania sposobem 2'].sum(), df_split[1]['Punkty z oceniania sposobem 2'].sum()]})
  sums["Grupa"] = i+4
  sums_list.append(sums)

df_concat = pd.concat(sums_list)

fig = px.bar(df_concat, x="połowa", y="Punkty z oceniania sposobem 2", facet_col="Grupa", color="połowa", text_auto=True, width=1500)

for i in range(df_concat['Grupa'].nunique()):
  pos = i * 2 + 1
  percent_change = 100*(df_concat.iloc[pos,1] - df_concat.iloc[pos-1,1]) / df_concat.iloc[pos-1,1]
  fig.add_annotation(x=1, y=df_concat.iloc[pos,1], text=f"{percent_change:.2f}%", showarrow=False, yshift=30, font=dict(color="red",size=14), xref=f'x{i+1}')

fig.update_xaxes(title_text=None)
fig.update_layout(showlegend=False, width=600, height=400, margin=dict(l=20, r=20, t=30, b=10))
fig.show(config=config)

Okazuje się, że sumy punktów dla drugiej połowy sekcji były mniejsze również w systemie oceniania, który mieszał kolejność prac. W 4 oraz 6 grupie jednak różnice te były zauważalnie mniejsze, natomiast w 5 grupie różnica ta była jeszcze większa. Mając takie dane ciężko więc jednoznacznie stwierdzić, czy kolejność prac sprawiła, że studenci byly skłonni do przydziału większej liczby punktów dla pierwszej połowy prac, jeśli w jednej z grup po przemieszaniu prac uzyskano jeszcze większą liczbę punktów przydzielonych dla pierwszej połówki projektów. Z uwagi na to, że grupa ta zawierała jednak zdecydowanie mniej uczestników niż pozostałe, nadal istnieje szansa na to, że zazwyczaj występuje faworyzacja projektów ocenianych jako pierwsze, w sczególności, gdy uczniowie mają ich do oceny bardzo dużo.


Interesującą kwestią dla autora pracy były najlepiej ocenione projekty według różnych systemów oceny. Postanowiono więc porównać pięć najlepszych projektów ocenionych według różnych sposobów w każdej grupie.

In [22]:
for i, data in enumerate((df_results_4, df_results_5, df_results_6), 4):
  print(f'Grupa {i}')
  first = (data.nlargest(5,'Punkty z oceniania sposobem 1'))[['Tytuł projektu', 'Punkty z oceniania sposobem 1']]
  second = (data.nlargest(5,'Punkty z oceniania sposobem 2'))[['Tytuł projektu', 'Punkty z oceniania sposobem 2']]
  print(f"Najlepiej ocenione sekcje sposobem 1:\n{[sekcja for sekcja in first.index]}")
  print(f"Najlepiej ocenione sekcje sposobem 2:\n{[sekcja for sekcja in second.index]}")
  # print('-'*100)
  print('\n')

Grupa 4
Najlepiej ocenione sekcje sposobem 1:
['Sekcja 9', 'Sekcja 1', 'Sekcja 3', 'Sekcja 29', 'Sekcja 38']
Najlepiej ocenione sekcje sposobem 2:
['Sekcja 5', 'Sekcja 23', 'Sekcja 3', 'Sekcja 11', 'Sekcja 29']


Grupa 5
Najlepiej ocenione sekcje sposobem 1:
['Sekcja 2', 'Sekcja 10', 'Sekcja 1', 'Sekcja 5', 'Sekcja 6']
Najlepiej ocenione sekcje sposobem 2:
['Sekcja 6', 'Sekcja 2', 'Sekcja 3', 'Sekcja 10', 'Sekcja 1']


Grupa 6
Najlepiej ocenione sekcje sposobem 1:
['Sekcja 3', 'Sekcja 5', 'Sekcja 1', 'Sekcja 2', 'Sekcja 9']
Najlepiej ocenione sekcje sposobem 2:
['Sekcja 3', 'Sekcja 26', 'Sekcja 19', 'Sekcja 2', 'Sekcja 9']




Wyniki oceniania oboma sposobami częściowo się pokrywały. Poniżej przedstawiono, ile projektów znalazło się w najlepszej piątce dla pierwszej i drugiej metody oceniania, dla drugiej i trzeciej oraz dla wszystkich trzech łącznie. Trzecią metodę oceniania wykorzystano w tych samych grupach, w których użyto metody drugiej.

In [23]:
rows_num = 5
for i, data in enumerate((df_results_4, df_results_5, df_results_6), 4):
  first = (data.nlargest(rows_num,'Punkty z oceniania sposobem 1'))[['Tytuł projektu', 'Punkty z oceniania sposobem 1']]
  second = (data.nlargest(rows_num,'Punkty z oceniania sposobem 2'))[['Tytuł projektu', 'Punkty z oceniania sposobem 2']]
  df_merged = pd.merge(first, second)
  print(f"Grupa {i} (Sposób 1 i 2): {' '*3}{len(df_merged)}/{rows_num}")

  data['Punkty z oceniania sposobem 3'] = data['Punkty z oceniania sposobem 3'].map(lambda x: x.replace('/3', ''))
  data = data.astype({'Punkty z oceniania sposobem 3': 'float'})
  third = (data.nlargest(rows_num,'Punkty z oceniania sposobem 3'))[['Tytuł projektu', 'Punkty z oceniania sposobem 3']]
  df_merged = pd.merge(second, third)
  print(f"Grupa {i} (Sposób 2 i 3): {' '*3}{len(df_merged)}/{rows_num}")

  df_merged = pd.merge(pd.merge(first, second), third)
  print(f"Grupa {i} (Sposób 1, 2 i 3): {len(df_merged)}/{rows_num}")
  print('-'*30)

Grupa 4 (Sposób 1 i 2):    2/5
Grupa 4 (Sposób 2 i 3):    3/5
Grupa 4 (Sposób 1, 2 i 3): 1/5
------------------------------
Grupa 5 (Sposób 1 i 2):    4/5
Grupa 5 (Sposób 2 i 3):    4/5
Grupa 5 (Sposób 1, 2 i 3): 3/5
------------------------------
Grupa 6 (Sposób 1 i 2):    3/5
Grupa 6 (Sposób 2 i 3):    2/5
Grupa 6 (Sposób 1, 2 i 3): 1/5
------------------------------


Zauważono, że niektóre z najlepiej punktowanych prac oceniono najlepiej niezależnie od metody oceniania. Liczba tych prac była mniejsza, kiedy porównano wyniki z trzech sposobów oceny, a nie tylko z dwóch. Biorąc pod uwagę, że osoby oceniające 3 sposobem przyznawały dodatkowe wyróżnienia pracom, sprawdzono, czy
prace które otrzymały najwięcej dodatkowych wyróżnień to również te, które dostały najwięcej punktów.

In [24]:
rows_num = 5
for i, data in enumerate((df_results_4, df_results_5, df_results_6), 4):
  data['Punkty z oceniania sposobem 3'] = data['Punkty z oceniania sposobem 3'].map(lambda x: x.replace('/3', ''))
  data = data.astype({'Punkty z oceniania sposobem 3': 'float'})
  third_points = (data.nlargest(rows_num,'Punkty z oceniania sposobem 3'))[['Tytuł projektu', 'Punkty z oceniania sposobem 3']]
  third_distinction = (data.nlargest(rows_num,'Wyróżnienia (studenci)'))[['Tytuł projektu', 'Wyróżnienia (studenci)']]
  df_merged = pd.merge(third_points, third_distinction)
  print(f'Grupa {i} - {len(df_merged)}/{rows_num}')
  # print('-'*30)

Grupa 4 - 3/5
Grupa 5 - 4/5
Grupa 6 - 1/5


Studenci mieli możliwość wyróżnienia dowolnych projektów, niekoniecznie tych, które oceniali najlepiej. Wyróżnienie można było przyznać na przykład za oryginalny pomysł, a nie za całokształt pracy. Analiza pokazuje jednak, że dla grupy czwartej oraz piątej większość z pięciu najlepiej ocenianych projektów miała też najwięcej wyróżnień.

In [25]:
# generowanie HTML/PDF (brak wykresów w pliku wyjściowym)
# !jupyter nbconvert --to HTML '/content/drive/MyDrive/Colab Notebooks/Draft-Magisterka.ipynb'
# KALEIDO package missing?

## Ankieta

In [26]:
# TEMP - Ankieta - revert changes to initial dataset (without loading file from drive)
# df_form = form.copy()

In [27]:
# wymiar danych
df_form.shape

(68, 119)

Sprawdzając wymiar danych, można zobaczyć, że ankieta została wypełniona 68 razy. Liczba 119 wskazuje, że możliwe jest występowanie nadmiarowych danych. Należało sprawdzić, czy nie występują tutaj niepożądane pytania ankietowe (kolumny zbioru). Wiadomo było, że odpowiedzi na niektóre pytania mogą być puste, gdyż 3 pytania w ankiecie były nieobowiązkowe. Sprawdzone zostaje więc, czy występują kolumny całkowicie puste. Poniższy kod pokazuje, ile procentowo stanowią odpowiedzi puste dla poszczególnych pytań, gdzie występuje przynajmniej jedna pusta odpowiedź.

In [28]:
def empty_percentage(df_form):
  columns_with_nulls = df_form[df_form.columns[df_form.isnull().any()]].isnull().sum() * 100 / df_form.shape[0]
  return columns_with_nulls.sort_values(ascending=False)

empty_percentage(df_form)

Pytanie                                                                                                                                        100.000000
Starałem/am się jak najbardziej pomóc w ulepszeniu/poprawieniu pracy formułując konkretne uwagi w ramach informacji zwrotnej2                  100.000000
Na ile trudne było dla Ciebie ocenianie kolegów/koleżanek?2                                                                                    100.000000
Oceń swoją satysfakcję z ocen swojej pracy otrzymanych od nauczyciela.2                                                                        100.000000
Na ile komfortowo czułeś(aś) się musząc wystawić ocenę kolegom/koleżankom?2                                                                    100.000000
                                                                                                                                                  ...    
Czy osoby oceniające powinny wiedzieć, kto jest autorem ocenianej pracy?2   

Wyraźnie widać, że zbiór posiada dużą liczbę pytań bez ani jednej odpowiedzi (puste wiersze stanowią 100% odpowiedzi). Po analizie tych pytań można zauważyć, że są to pytania, które nie pojawiły się w finalnej wersji ankiety, jednak narzędzie jakie było używane do stworzenia ankiety, zapisało je w wyjściowym pliku zawierającym wyniki. Te pytania można było więc usunąć ze zbioru, a następnie sprawdzić ile pozostało pytań z niekompletną liczbą odpowiedzi.

In [29]:
df_form.dropna(axis=1, how='all', inplace=True)
empty_percentage(df_form)

Czy znasz jakąś metodę oceny koleżeńskiej, którą chciałbyś/chciałabyś wykorzystać, oceniając prace rówieśników? Jeśli tak opisz ją poniżej.    79.411765
Na ile prawdopodobne jest, że znając autora recenzji skorzystałbyś/skorzystałabyś z możliwości bezpośredniego przedyskutowania z nim uwag?     79.411765
Czy masz jakieś dodatkowe uwagi lub przemyślenia?                                                                                              73.529412
Co można według Ciebie poprawić w aplikacji Zapunktuj?                                                                                         67.647059
dtype: float64

Po usunięciu niepożądanych pytań spodziewano się, że w zbiorze pozostaną tylko 3 pytania, na które odpowiedzi nie udzieliło 100% osób, ponieważ dokładnie tyle było nieobowiązkowych pytań. W zbiorze pojawiło się jednak jedno pytanie, na które odpowiedziało niewiele osób pomimo tego, że było ono obowiązkowe. Przyczyną był prawdopodobnie błąd techniczny. Mimo wielu brakujących odpowiedzi na to pytanie zdecydowano się nie usuwać go ze zbioru.

Po usunięciu pustych kolumn ich liczba zmniejszyła się z 119 do 57. Atrybutów w zbiorze jest więcej, niż wynosi liczba pytań w ankiecie. Jest tak między innymi dlatego, że niektóre z pytań były złożone, więc musiały zostać podzielone na kilka części.  

Nadal jednak w zbiorze znajdują się kolumny z niepożądanymi informacjami. Z uwagi na fakt, że postanowiono zbadać ankietę anonimowo, to usunięte zostaną z niej informacje zawierające dane osobiste. Są to kolumny Adres e-mail oraz Nazwa, która zawiera imię i nazwisko osoby wypełniającej formularz. Usunięte zostały też kolumny zawierające informacje o godzinie rozpoczęcia i godzinie zakończenia wypełniania ankiety.

In [30]:
# temp - to check results
# temp = df_form.copy()
# temp.drop(labels=['Godzina rozpoczęcia', 'Godzina ukończenia', 'Adres e-mail', 'Nazwa'], axis=1, inplace=True)
# temp.shape
df_form.drop(labels=['Godzina rozpoczęcia', 'Godzina ukończenia', 'Adres e-mail', 'Nazwa'], axis=1, inplace=True)

Następnym krokiem było przeformatowanie danych.
Kolumna ID wybrana zostaje jako index dla zbioru danych, ponieważ jej wartość jest unikalna dla każdego rekordu.
<!-- Wszystkim danym zmieniony zostaje typ na str. -->

In [31]:
df_form.set_index('ID', inplace=True)
# df_form = df_form.astype(str)

Z uwagi na to, że część pytań została podzielona na wiele części, konieczna była ich separacja ze zbioru. Sposób zapisu odpowiedzi na te pytania utrudniał ich dopasowanie do konkretnego pytania, gdyż treść pytania nie została zapisana do skoroszytu. Z pomocą uproszczonego zestawienia wyników wygenerowanego przez narzędzie Microsoft Forms wyodrębniono odpowiedzi i zapisano w osobnych zmiennych razem z treścią pytania.
<!-- Poniżej screen z pytaniem (z podglądu formularza - bez wyników) żeby pokazać w czy problem-->

In [32]:
def cut_columns(df, columns):
  new_df = df[columns]
  df.drop(columns, axis=1, inplace=True)
  return new_df

# 1
df_qst_1 = cut_columns(df_form, ["Ocena przez nauczyciela"," Koledzy + kontrola nauczyciela", "Zapunktuj - przydział punktów z jednej puli2",
                                 "Zapunktuj - przydział punktów w skali2", "Zapunktuj - zostawienie komentarzy w formie \"Two stars and a wish\"2"])
df_qst_1.question = "Określ, na ile obiektywna (twoim zdaniem) jest ocena w przypadku różnych sposobów oceniania"

# 3
df_qst_3 = cut_columns(df_form, ["Warsztaty Moodle", "Zapunktuj - przydział punktów z jednej puli",
                                 "Zapunktuj - przydział punktów w skali", "Zapunktuj - zostawienie komentarzy w formie \"Two stars and a wish\""])
df_qst_3.question = "Czy i na ile odpowiadają Ci poszczególne metody oceny koleżeńskiej?"

# 7
df_qst_7 = cut_columns(df_form, ['Sformułowanie informacji zwrotnej dla prac kolegów/koleżanek', 'Obsługa Zapunktuj', 'Obsługa warsztatów w Moodle'])
df_qst_7.question = "Jak bardzo kłopotliwe było dla Ciebie:"

# 25
df_qst_25 = cut_columns(df_form, ['W pewnym stopniu się przydały', 'Były dla mnie niezasadne, a przez to nieprzydatne',
                                  'Były dla mnie niezrozumiałe, a przez to nieprzydatne', 'Zdecydowanie pomogły'])
df_qst_25.question = "W jakim stopniu informacje zwrotne uzyskane od kolegów/koleżanek pomogły Ci zauważyć potencjalne niedoskonałości Twojej pracy?"

# 28
df_qst_28 = cut_columns(df_form, ['Udało mi się zauważyć trochę rzeczy, które zrobiłbym/zrobiłabym inaczej',
                                  'Starałem/am się "podpatrzeć" co koledzy/koleżanki zrobili lepiej, by w przyszłości skorzystać z ich pomysłu/podejścia',
                                  'Starałem/am się jak najbardziej pomóc w ulepszeniu/poprawieniu pracy formułując konkretne uwagi w ramach informacji zwrotnej',
                                  'Czy to, że miałeś/as wgląd w kilka prac dotyczących tego samego zadania pomogło Ci inaczej spojrzeć na temat, sposób opracowania?',
                                  'Czy starałeś/aś się podejść do oceny prac kolegów/koleżanek rzetelnie i poważnie?'])
df_qst_28.question = "Jak oceniasz swoje podejście do oceny prac kolegów/koleżanek?"

# 29
df_qst_29 = cut_columns(df_form, ['Jako jedynej oceny - końcowej za zadanie',
                                  'Jako oceny "pośredniej", pozwalającej otrzymać informacje zwrotne umożliwiające poprawienie zadania i wyeliminowanie potencjalnych błędów PRZED przesłaniem pracy do oceny końcowej?',
                                  'Jako oceny końcowej, ale odbywającej się niejako w dwóch iteracjach - pierwszej do zebrania uwag i zastrzeżeń i drugiej - do oceny końcowej'])
df_qst_29.question = "Jak oceniasz przydatność oceny koleżeńskiej"

Można było także wyodrębnić pytania nieobowiązkowe.

In [33]:
# 32, 33, 34
df_qst_optional = cut_columns(df_form, ["Co można według Ciebie poprawić w aplikacji Zapunktuj?",
                                        "Czy znasz jakąś metodę oceny koleżeńskiej, którą chciałbyś/chciałabyś wykorzystać, oceniając prace rówieśników? Jeśli tak opisz ją poniżej.",
                                        "Czy masz jakieś dodatkowe uwagi lub przemyślenia?"])

Niektóre z odpowiedzi zawierały niepożądane znaki liczbowe. Pojawiły się one z uwagi na to, że niektóre z odpowiedzi powtarzały się między różnymi pytaniami. Znaki te zostały usunięte.
<!-- Poniżej usuwanie tych znaków z wyodrębnionych pytań/odpowiedzi -->

In [34]:
questions = list([df_form, df_qst_1, df_qst_3, df_qst_7, df_qst_25, df_qst_28, df_qst_29, df_qst_optional])

for el in questions:
  df_qst_1.rename(mapper=(lambda x: x.rstrip(digits)), axis='columns', inplace=True)

Poprawiono też treść jednego z pytań.

In [35]:
old_name = "Czy uważasz, że sporządzenie dostępnego dla wszystkich rankingu prac według przyznanych punktów (przez kolegów/koleżanki/nauczyciela) to dobre rozwiązanie, czy raczej oceny projektów oraz komentar..."
new_name = "Czy uważasz, że sporządzenie dostępnego dla wszystkich rankingu prac według przyznanych punktów (przez kolegów/koleżanki/nauczyciela) to dobre rozwiązanie, czy raczej oceny projektów oraz komentarze powinny być widoczne tylko dla autorów pracy?"
df_form.rename(columns={old_name: new_name}, inplace=True)

Po wstępnej obróbce danych można było przejść do analizy odpowiedzi w ankiecie.

<!-- ### Ogólne wrażenia dotyczące oceny rówieśniczej -->
### Opinie na temat otrzymanych informacji zwrotnych

<!-- Na początku zdecydowano się sprawdzić, jakie opinie mieli studenci na temat wzajemnej oceny oraz jak oceniają otrzymane informacje zwrotne od rówieśników. -->

Na początku zdecydowano się sprawdzić, czy studenci postrzegają rezultaty przeprowadzonej oceny wzajemnej bardziej pozytywnie, czy też negatywnie. W jednym z pytań studenci mieli ocenić w skali od 1 do 5 swoją satysfakcję z otrzymanych ocen.
<!-- jak studenci oceniają otrzymane oceny oraz informacje zwrotne od rówieśników.  -->


In [36]:
def string_divider(sequence, interval=0, sep='<br>', index=0, skip_short=False, intervals=list()):
  str_split = sequence.split(' ')
  if not intervals:
    if index > 0:
      return sequence[:index] + sep + sequence[index:]
    if interval == 0:
      interval = len(str_split) // 2
    if skip_short is True and len(str_split) <= interval:
      return sequence
    else:
      return ' '.join(str_split[0:interval]) + sep + ' '.join(str_split[interval:len(str_split)])
  else:
    input = sequence
    for i in intervals:
      str_split = input.split(' ')
      if skip_short is True and len(str_split) <= i:
        continue
      else:
        input = ' '.join(str_split[0:i]) + sep + ' '.join(str_split[i:len(str_split)])
    return input

In [37]:
#15
qst_15 = df_form.filter(like='Oceń swoją satysfakcję', axis=1).iloc[:,0]
qst_15_counts = qst_15.value_counts()
qst_15_counts.index = qst_15_counts.index.map(str)
fig = px.bar(qst_15_counts.sort_values(), title=string_divider(qst_15_counts.name), text_auto=True, labels={ "index": "ocena", "value": "liczba głosów" }, width=800, color=qst_15_counts.index)
fig.update_layout(showlegend=False, width=600, height=400, margin=dict(l=20, r=20, t=70, b=10))
fig.add_annotation(x=2, y=20, xref="x", yref="y", text=f"średnia={qst_15.mean():.1f}", showarrow=False, font=dict(family="Courier New, monospace", size=16, color="#ffffff"),
                   align="center",bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#ff7f0e",)
fig.show(config=config)

Na wykresie widać, że większość osób była zadowolona z ocen otrzymanych od rówieśników, jednak nie brakło również takich osób, które nie były usatysfakcjonowane. Średnia satysfakcja z ocen wyniosła 3.8.

W następnym z pytań studenci ocenili przydatność otrzymanych informacji zwrotnych od swoich rówieśników. Wykres pokazuje, że większość studentów postrzegała informacje zwrotne jako przydatne.
<!-- W następnym z pytań studenci ocenili rzetelność otrzymanych informacji zwrotnych od swoich rówieśników. Wykres pokazuje, że większość studentów postrzegała informacje zwrotne jako rzetelne. -->

In [38]:
qst_21 = df_form.filter(like='Jak oceniasz informacje zwrotne', axis=1)
counts = qst_21.groupby(qst_21.columns[0]).size()
counts = counts.reset_index(name="liczba")
fig = px.pie(counts, values="liczba", names=qst_21.columns[0], title=string_divider(counts.columns[0]), width=700)
fig.update_layout(width=620, height=400, margin=dict(l=20, r=20, t=70, b=10))
fig.show(config=config)

Co więcej, większość studentów uważała także, że otrzymane informacje zwrotne zostały sformułowane rzetelnie.

In [39]:
qst_16 = df_form.filter(like='Czy informacje zwrotne otrzymane', axis=1)
counts = qst_16.groupby(qst_16.columns[0]).size().reset_index(name="liczba")
fig = px.pie(counts, values="liczba", names=qst_16.columns[0], title=string_divider(counts.columns[0],index=48), width=500)
fig.update_layout(width=500, height=400, margin=dict(l=20, r=20, t=70, b=10))
fig.show(config=config)

Poruszono też kwestię funkcji, jaką pełnić może ocena koleżeńska po zrealizowaniu zadania. Studentów zapytano, jak oceniają przydatność takiej oceny w różnych możliwych scenariuszach jej użycia.

In [40]:
color_palette = px.colors.sequential.Inferno
color_palette_name = 'inferno'
def get_palette_discretes(start=2, end=len(color_palette)-1, step=3):
    return color_palette[start:end:step]

In [41]:
qst_29 = df_qst_29.apply(pd.Series.value_counts).transpose()
qst_29.rename(index={qst_29.index.values[1]: string_divider(qst_29.index.values[1], intervals=[3,5,7,9,11], sep='<br>')}, inplace=True)
qst_29.rename(index={qst_29.index.values[2]: string_divider(qst_29.index.values[2], intervals=[5,8,13])}, inplace=True)
qst_29 = qst_29[['Nieprzydatna', 'Raczej nieprzydatna', 'Neutralna', 'Raczej przydatna', 'Przydatna']]
fig = px.bar(qst_29.sort_values(by='Przydatna'), title=string_divider(df_qst_29.question, interval=3), orientation='h', width=800,
            #  color_discrete_sequence=get_palette_discretes(start=3,step=1),
             labels={ "index": "pytanie", "value": "liczba odpowiedzi", "variable": "legenda"}, text_auto=True)


fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02,  xanchor="right", x=1), margin=dict(l=10, r=0, t=10, b=10))
fig.show(config=config)

Odpowiedzi pokazują, że najchętniej studenci wykorzystaliby ocenianie rówieśnicze do zebrania uwag dotyczących swojej pracy, co umożliwiłoby poprawę zadania przed wystawieniem finalnej oceny. Zdecydowanie mniej entuzjastycznie studenci podeszli do pomysłu wykorzystania oceny rówieśniczej jako oceny końcowej. Może to sugerować, że studenci mają większe zaufanie w tej kwestii do nauczyciela, natomiast nadal postrzegają ocenianie wzajemne jako użyteczny proces. Potwierdzają to odpowiedzi na inne pytania, gdzie wprost zapytano, czyja ocena jest bardziej preferowana i ma większą wartość.

In [42]:
qst_31 = df_form.filter(like='Jeśli miałbyś/miałabyś wybór', axis=1)
counts_31 = qst_31.groupby(qst_31.columns[0]).size().reset_index(name="liczba")

qst_13 = df_form.filter(like='Która z ocen ma dla Ciebie większą', axis=1)
counts_13 = qst_13.groupby(qst_13.columns[0]).size().reset_index(name="liczba")

counts_31.iloc[:,0] = counts_31.iloc[:,0].apply(lambda x : string_divider(x,interval=3, skip_short=True))
counts_13.iloc[:,0] = counts_13.iloc[:,0].apply(lambda x : string_divider(x,interval=3, skip_short=True))

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=[string_divider(counts_31.columns[0], intervals=[2,6]), string_divider(counts_13.columns[0], intervals=[4,10])])

fig.add_trace(go.Pie(labels=counts_31.iloc[:,0], values = counts_31["liczba"], legendgroup = '1', ), 1, 1)
fig.add_trace(go.Pie(labels=counts_13.iloc[:,0], values = counts_13["liczba"], legendgroup = '2', ), 1, 2)

fig.update_layout(autosize=False, width=800, height=400, legend_tracegroupgap=60)
fig.update_layout(margin=dict(l=10, r=20, t=50, b=0), legend=dict(y=0.5))
for annotation in fig['layout']['annotations']:
        annotation['y']=0.93

fig.show(config=config)

Na wykresie widać, że większość studentów preferuje przeprowadzenie oceny zarówno przez nauczyciela, jak i przez rówieśników. Blisko 70% osób postrzega ocenę od nauczyciela jako bardziej wartościową.

### Anonimowość w procesie oceny wzajemnej

Dość istotnym elementem w przeprowadzaniu oceniania rówieśniczego jest kwestia jej anonimowości. Przy projektowaniu aplikacji Zapunktuj postanowiono, aby wystawiane oceny i komentarze nie miały przypisanego autora, więc były one anonimowe. Mimo że aplikacja nie wymagała podpisania swojej pracy, część studentów wpisała swoje nazwiska w tytule pracy lub jej opisie. Przez to nie udało się całkowicie ukryć, kto był autorem pracy. W aplikacji Moodle wyglądało to podobnie. Oceny oraz informacje zwrotne również były anonimowe, natomiast informacja o autorze pracy była jawna i zawsze było wiadomo, czyja praca jest oceniana.

W ankiecie postanowiono zapytać o preferencje związane z anonimowością oceny, aby przekonać się, czy rozwiązania w aplikacjach Zapunktuj oraz Moodle odpowiadają oczekiwaniom studentów.

In [43]:
qst_18 = df_form.filter(like='Czy osoby oceniające powinny wiedzieć', axis=1)
counts_18 = qst_18.groupby(qst_18.columns[0]).size().reset_index(name="liczba")

qst_19 = df_form.filter(like='Czy osoby oceniane powinny wiedzieć,', axis=1)
qst_19.replace('np', 'np.', regex=True, inplace=True)
counts_19 = qst_19.groupby(qst_19.columns[0]).size().reset_index(name="liczba")

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=[string_divider(counts_18.columns[0], intervals=[4,7]), string_divider(counts_19.columns[0], intervals=[4,7]).replace(' ?','?')])

counts_18.iloc[:,0] = counts_18.iloc[:,0].apply((lambda x : string_divider(x, intervals=[7,8], skip_short=True).replace('(np. ','').replace('wa)','wa')))
counts_19.iloc[:,0] = counts_19.iloc[:,0].apply((lambda x : string_divider(x, intervals=[7,8], skip_short=True).replace('(np. ','').replace('wa)','wa')))

fig.add_trace(go.Pie(labels=counts_18.iloc[:,0], values = counts_18["liczba"],), 1, 1)
fig.add_trace(go.Pie(labels=counts_19.iloc[:,0], values = counts_19["liczba"],), 1, 2)

fig.update_layout(autosize=False, width=800, height=400, legend=dict(y=0.45))
fig.update_layout(margin=dict(l=10, r=20, t=40, b=0), legend=dict(y=0.5))

for annotation in fig['layout']['annotations']:
        annotation['y']=0.88

fig.show(config=config)

Większość studentów jest zdania, że zarówno autorzy pracy, jak i autorzy recenzji powinni pozostać anonimowi. Jest to sygnał do tego, aby w przyszłości dopracować kwestię anonimowości autorów prac. W przypadku aplikacji Zapunktuj, można studentów poinformować, aby nie podpisywali swoich prac oraz w razie potrzeby usunąć je przed fazą oceniania, jeśli takie podpisy się pojawią.
Zastanawiano się także, czy same oceny i informacje zwrotne powinny być widoczne dla każdego. W aplikacji Zapunktuj przedstawiany jest ranking prac, gdzie każdy może zobaczyć, jak została oceniona praca oraz jakie komentarze otrzymała. Studentów zapytano więc, czy takie rozwiązanie im odpowiada.

In [44]:
pd.set_option('max_colwidth', None)
qst_6 = df_form.filter(like='Czy uważasz, że sporządzenie dostępnego dla wszystkich rankingu prac według przyznanych', axis=1)
counts_6 = qst_6.groupby(qst_6.columns[0]).size().reset_index(name="liczba")
counts_6.iloc[:,0] = counts_6.iloc[:,0].apply(lambda x : string_divider(x,intervals=[5,9], skip_short=True))
fig = px.pie(counts_6, values="liczba", names=qst_6.columns[0], title=string_divider(counts_6.columns[0],intervals=[9,14,22]), width=800, height=400)
fig.update_layout(legend=dict(y=0.5), title_y=0.95, margin=dict(l=10, r=20, t=100, b=10))

fig.show(config=config)

Około 20% osób uważa dostępność rankingu w obecnej formie za dobre rozwiązanie, jednak największa grupa osób jest zdania, że oceny i uwagi w formie komentarzy nie powinny być dostępne dla wszystkich. W takim wypadku należało przemyśleć, jak można zmienić formę rankingu lub ograniczyć widoczność poszczególnych elementów tak, aby widoczne były tylko dla autorów prac. Zastanawiano się także, czy świadomość tego, że prace zostaną zobaczone przez rówieśników, wpłynie w jakiś sposób na motywację do lepszego wykonania pracy. Spodziewano się, że studenci będą czuli w związku z tym dodatkową presję.

In [45]:
qst_23 = df_form.filter(like='Czy świadomość tego, że koledzy i koleżanki zobaczą twoją pracę', axis=1)
counts = qst_23.groupby(qst_23.columns[0]).size().reset_index(name="liczba")
fig = px.pie(counts, values="liczba", names=qst_23.columns[0], title=string_divider(counts.columns[0]), width=700)

fig.update_layout(width=800, height=400, legend=dict(y=0.5))
fig.update_layout(margin=dict(l=10, r=10, t=60, b=10), title_y=0.93)

fig.show(config=config)

Badanie pokazało, że dla większości osób fakt zobaczenia pracy przez kolegów i koleżanki nie miał większego znaczenia. Jednak niemalże 37% osób, odczuło dodatkową motywację.

### Trudności związane z oceną wzajemną

Z uwagi na to, że proces oceny wzajemnej może wiązać się z wieloma trudnościami, postanowiono zapytać studentów o te kłopotliwe kwestie. Zapytano między innymi jak trudne ogółem, było ocenianie rówieśników.




In [46]:
qst_26 = df_form.filter(like='Na ile trudne było dla Ciebie ocenianie', axis=1)
counts_26 = qst_26.groupby(qst_26.columns[0]).size().reset_index(name="liczba")
counts_26.iloc[:,0] = counts_26.iloc[:,0].apply(lambda x : string_divider(x,interval=4, skip_short=True))
fig = px.pie(counts_26, values="liczba", names=qst_26.columns[0], title=counts_26.columns[0], width=700)
fig.update_layout(width=700, height=300, legend=dict(y=0.5))
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10), title_y=0.95)

fig.show(config=config)

Wykres pokazał, że większość studentów dostrzegła jakieś trudności podczas oceniania. Zastanawiano się więc, które aspekty mogły być najbardziej problematyczne. W jednym z pytań zapytano o to, jakie kwestie były najmniej komfortowe podczas oceny.

<!-- Zastanawiano się czy problematyczne było dla studentów wystawianie oceny, formułowanie informacji zwrotnej czy poświęcenie dodatkowego czasu na recenzję prac rówieśników. W jednym z pytań zapytano, które z tych trzech kwestii była najmniej komfortowa. -->

In [47]:
qst_9 = df_form.filter(like='Co jest dla Ciebie najmniej komfortowe', axis=1)
counts_9 = qst_9.groupby(qst_9.columns[0]).size().reset_index(name="liczba")
counts_9.iloc[:,0] = counts_9.iloc[:,0].apply(lambda x : string_divider(x,interval=4, skip_short=True))
fig = px.pie(counts_9, values="liczba", names=qst_9.columns[0], title=counts_9.columns[0], width=700)
fig.update_layout(legend=dict(y=0.5), title_y=0.82)

fig.update_layout(width=700, height=300, legend=dict(y=0.5))
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10), title_y=0.95)

fig.show(config=config)

Wykres pokazuje, że najmniej komfortowym elementem podczas oceny koleżeńskiej była konieczność poświęcenia dodatkowego czasu na recenzję prac. Dla mniejszej liczby osób najmniej komfortowa była sama ocena, czy obawy przed reakcją rówieśników. Może to sugerować, że liczba prac jaką musieli zrecenzować studenci, była za duża. W ankiecie poruszono ten problem, pytając studentów o optymalną liczbę prac do oceny.

In [48]:
qst_10 = (df_form.filter(like='Ile prac według Ciebie powinna oceniać jedna osoba', axis=1)).astype(str)
counts_10 = qst_10.groupby(qst_10.columns[0]).size().reset_index(name="liczba")
counts_10.iloc[:,0] = counts_10.iloc[:,0].apply(lambda x : x[5:10].replace('0','') if len(x)==19 else x)

qst_11 = (df_form.filter(like='Po przejrzeniu ilu prac koleżanek/kolegów podczas oceny', axis=1)).astype(str)
counts_11 = qst_11.groupby(qst_11.columns[0]).size().reset_index(name="liczba")
counts_11.iloc[:,0] = counts_11.iloc[:,0].apply(lambda x : x[5:10].replace('0','') if len(x)==19 else x)

counts_10.replace('To zależy od ocenianiej pracy (i czasu który trzeba poświecić na zapoznanie się z pracą)','To zależy od zadania/pracy', inplace=True)

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=[string_divider(counts_10.columns[0],intervals=[5,8]),
                                    string_divider(counts_11.columns[0], intervals=[5,8,12])+'?'])

counts_10.iloc[:,0] = counts_10.iloc[:,0].apply((lambda x : string_divider(x, interval=8, skip_short=True)))
counts_11.iloc[:,0] = counts_11.iloc[:,0].apply((lambda x : string_divider(x, interval=8, skip_short=True)))

fig.add_trace(go.Pie(labels=counts_10.iloc[:,0], values=counts_10["liczba"], ), 1, 1)
fig.add_trace(go.Pie(labels=counts_11.iloc[:,0], values=counts_11["liczba"], ), 1, 2)

fig.update_layout(autosize=False, width=1100, legend=dict(y=0.5))

fig.update_layout(width=850, height=400, legend=dict(y=0.5))
fig.update_layout(margin=dict(l=10, r=10, t=90, b=10), title_y=0.90)

fig.show(config=config)

Według połowy ankietowanych liczba prac do oceny przypadająca na jedną osobę powinna zależeć od rodzaju tej pracy. Około 44% osób uważa, że liczba prac powinna wynosić od 1 do 6. Ledwie kilka osób preferowało 7 lub więcej prac.

Na prawym wykresie można zauważyć, że odpowiedzi wyglądały bardzo podobnie. Według studentów wystarczająca liczba prac pozwalająca zauważyć elementy do poprawy jest podobna do liczby prac, jaką ogółem powinno się przyznać na jedną osobę do ewaluacji.

Z uwagi na to, że w aplikacji Zapunktuj liczba prac do oceny wynosiła od kilku do nawet kilkudziesięciu, można przypuścić, że liczba ta była w niektórych przypadkach za duża, co mogłoby tłumaczyć, dlaczego tak wiele osób postrzegało konieczność poświęcenia dodatkowego czasu za najmniej komfortowy czynnik podczas oceny wzajemnej. W odpowiedziach na pytania otwarte zwracano uwagę na ten aspekt.

> "Za dużo zadań do oceny[...]"

> "[...]przegląd dużej ilości prac był trochę nieczytelny[...]"

Tak duża liczba prac do oceny przypadająca na jedną osobę w autorskiej aplikacji była jednak spowodowana przez implementację sposobów oceniania polegających przydział punktów z jednej puli. Aby punktacja była sprawiedliwa, konieczne było wyświetlenie wszystkich prac do oceny. W takim przypadku należało się zastanowić, czy z uwagi na wadę tego systemu nie należy go wyeliminować i zastąpić innym sposobem oceny pozbawionym tej wady - na przykład już istniejącym sposobem trzecim (jako jedynym w systemie). Porównanie różnych metod oceny na podstawie odpowiedzi z ankiety zostanie jeszcze przedstawione w dalszej części pracy. Z uwagi na to, że wiele osób uzależnia optymalną liczbę prac do oceny od typu zadania, pokazane zostaną typy prac oceniane przez ankietowanych. Pytanie było wielokrotnego wyboru, ponieważ niektórzy oceniali więcej niż jeden typ pracy (na różnych platformach). Odpowiedzi o niewielkim udziale procentowym zostały pominięte.

In [49]:
qst_24 = df_form.filter(like='Jakie typy prac', axis=1)
qst_24.iloc[:, 0] = qst_24.iloc[:, 0].apply(lambda x : x.lower())
qst_24.replace('loga','logo', regex=True, inplace=True)
df_24 = (qst_24.iloc[:, 0].str.get_dummies(';'))
df_24 = df_24.apply(pd.Series.value_counts).transpose()
df_24.rename(columns = {0:'nie', 1:'tak'}, inplace = True)
df_24 = df_24.sort_values(by=['tak'], ascending=False)
df_24['suma'] = df_24['tak'] / len(qst_24.index)
df_24['suma'] = df_24['suma'].apply(lambda x : f'{(x*100):.1f}%')
df_24.index.name = 'projekt'
df_24.head(5)

,nie,tak,suma
projekt,,,
infografika,1,67,98.5%
logo,50,18,26.5%
kod programu,53,15,22.1%
praca pisemna,61,7,10.3%
notatnik jupyter (kod programu + pisemny opis),66,2,2.9%


Infografika była typem pracy, który oceniali niemalże wszyscy. Sporo osób oceniło także logo, kod programu, pracę pisemną czy notatnik Jupyter.

Innym aspektem, który mógłby sprawiać problemy w procesie oceniania, są kwestie interpersonalne. Niezależnie od tego czy ocena jest, czy nie jest anonimowa, podejrzewano, że świadomość oceniania swoich rówieśników może być dla uczniów kłopotliwa. Zapytano więc o to, czy relacje w grupie miały znaczenie.

In [50]:
qst_2 = df_form.filter(like='Zaznacz odpowiedzi, z którymi się zgadzasz', axis=1)
df_qst_2 = (qst_2.iloc[:, 0].str.get_dummies(';')).apply(pd.Series.value_counts).transpose()
df_qst_2.rename(columns = {0:'Nie zgadzam się', 1:'Zgadzam się'}, inplace = True)
df_qst_2.index.name = 'kwestia'
df_qst_2_4 = df_qst_2.loc['Relacje wewnątrz grupy i personalne mają wpływ na ocenę']

qst_27 = df_form.filter(like='Na ile komfortowo czułeś(aś)', axis=1)
counts_27 = qst_27.groupby(qst_27.columns[0]).size().reset_index(name="liczba")

fig_sub = make_subplots(rows=1, cols=2, specs=[[{"type": "bar"}, {"type": "pie"}]],
                        subplot_titles=[string_divider(df_qst_2_4.name, interval=5),
                                        string_divider(counts_27.columns[0], interval=6)])

fig_sub.add_trace(go.Bar(x=['Odpowiedzi'], y=df_qst_2_4[:1], text=(df_qst_2_4.index[0] + f"{'<br>'*2}{df_qst_2_4[0]/len(qst_2.index)*100:.1f}%"), showlegend=False), 1, 1)
fig_sub.add_bar(x=['Odpowiedzi'], y=df_qst_2_4[1:], text=(df_qst_2_4.index[1] + f"{'<br>'*2}{df_qst_2_4[1]/len(qst_2.index)*100:.1f}%"), showlegend=False)

fig_sub.add_trace(go.Pie(labels=counts_27.iloc[:,0], values = counts_27["liczba"], ),1,2)
fig_sub.update_layout(legend=dict(y=0.5), width=1100, title=dict(x=0.1), barmode='stack')
for annotation in fig_sub['layout']['annotations']:
        annotation['y']=1.03


fig_sub.update_layout(width=900, height=400, legend=dict(y=0.5))
fig_sub.update_layout(margin=dict(l=0, r=0, t=60, b=10), title_y=0.90)

fig_sub.show(config=config)

Większość osób uważała, że kwestie interpersonalne w grupie mają wpływ na oceny prac. Około 37% ankietowanych odczuło jakiś dyskomfort, wystawiając ocenę rówieśnikom, jednak większość nie miała z tym żadnego problemu.
<!-- Dla większości również wystawianie oceny rówieśnikom nie wiązało się z dyskomfortem, jednak około 37% odczuło jakiś dyskomfort -->

### Opinie o aplikacjach oraz użytych w nich metodach oceniania.

Jednym z powodów stworzenia aplikacji Zapunktuj była trudność w przeprowadzeniu oceny rówieśniczej stacjonarnie na uczelni bez wykorzystania jakiejkolwiek aplikacji komputerowej. Aplikacja w założeniu miała ułatwić ten proces i wyeliminować niektóre z wad. Zapytano więc studentów czy wykorzystanie takich platform ułatwia według nich przeprowadzenie oceny.

In [51]:
qst_22 = df_form.filter(like='Czy według Ciebie wykorzystanie platformy internetowej', axis=1).iloc[:,0]
counts_22 = qst_22.value_counts()
counts_22.loc['Utrudnia'] = 0
counts_22.loc['Bardzo utrudnia'] = 0
counts_22.rename(index=(lambda x : string_divider(x)), inplace=True)
fig = px.bar(counts_22, title=string_divider(counts_22.name), text_auto=True, color=counts_22.index, width=1000, labels={ "index": "opinia", "value": "liczba głosów" })
fig.update_layout(showlegend=False)
fig.update_layout(width=750, height=400, legend=dict(y=0.5))
fig.update_layout(margin=dict(l=10, r=10, t=80, b=10), title_y=0.90)

fig.show(config=config)

Kilka osób wyraziło neutralną opinię, jednak zdecydowana większość uważa, że wykorzystanie platformy internetowej ułatwia trochę lub znacząco przeprowadzanie oceny koleżeńskiej. Co więcej, nikt nie był zdania, że platformy te w jakikolwiek sposób utrudniają ten proces. Zapytano również o to, czy obsługa Zapunktuj lub Moodle sprawiała jakieś trudności.

In [52]:
qst_7 = df_qst_7.apply(pd.Series.value_counts).transpose()
qst_7.rename(index=(lambda x : string_divider(x)), inplace=True)

fig = px.bar(qst_7[1:], title=df_qst_7.question, orientation='h', height=400, width=700,
             labels={ "index": "pytanie", "value": "liczba odpowiedzi", "variable": "legenda"}, text_auto=True)

fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02,  xanchor="right", x=1), margin=dict(l=10, r=10, t=10, b=10))

fig.update_layout(width=750, height=400)
fig.update_layout(margin=dict(l=10, r=5, t=30, b=10), title_y=0.97)

fig.show(config=config)

Wykres pokazuje, że bardziej kłopotliwe było korzystanie z Zapunktuj, jednak większość osób nie miała żadnych problemów w korzystaniu z aplikacji.

Kolejnym tematem poruszanym w ankiecie były opinie o metodach oceny wykorzystywanych w aplikacjach.
<!-- można wspomnieć gdzieś jak wyglądało ocenianie w Moodle i przeanalizować odpowiedzi na pytanie 8 -->

In [53]:
qst_3 = df_qst_3.apply(pd.Series.value_counts).transpose().fillna(0)

qst_3 = qst_3.rename(index=(lambda x : string_divider(x, interval=4, skip_short=True)))
qst_3 = qst_3.rename(columns=(str.lstrip))

qst_3['Nie mam zdania lub nie oceniałem/am tą metodą'] = qst_3['Nie oceniałem/am tą metodą'] + qst_3['Nie mam zdania']

qst_3 = qst_3[['Zdecydowanie nie podoba się', 'Nie podoba mi się', 'Podoba się', 'Zdecydowanie podoba mi się']]
qst_3 = qst_3.rename(columns=(lambda x : string_divider(x, interval=2, skip_short=True)))
fig = px.bar(qst_3.sort_values(by='Zdecydowanie podoba<br>mi się'), title=string_divider(df_qst_3.question, interval=7), orientation='h', width=1100, height=400,
            #  color_discrete_sequence=get_palette_discretes(start=3,step=1),
             labels={"index": "metoda oceny", "value": "liczba odpowiedzi", "variable": "legenda"}, text_auto=True)

fig.update_layout(legend=dict(orientation="v", y=0.5),)

fig.update_layout(width=800, height=400, legend=dict(y=0.5))
fig.update_layout(margin=dict(l=10, r=5, t=55, b=10), title_y=0.95)

fig.show(config=config)

W większości wykorzystane sposoby oceniania odpowiadały studentom. Jeden z nich jednak wyróżnia się jednak dużą liczbą negatywnych opinii. Ocenianie przez przydział punktów z jednej puli w aplikacji Zapunktuj nie odpowiadał blisko połowie ankietowanych. W pytaniach otwartych niektórzy opisali dokładniej, dlaczego ten system oceniania nie przypadł im do gustu.

> "System punktowy z jednej puli- ilość punktów może zależeć nie od tego czy praca była dobrze wykonana ale ze względu na relacje między osobami.[...]"

> "Platforma Zapunktuj w formie pula punktów jest trochę nierzetelna z mojego punktu widzenia, ze względu na przyjętą przez każdego własną skalę bez odgórnych wytycznych, jest bardziej przydatna w kontekście informacji zwrotnej."

W innym pytaniu zapytano, które z metod są najbardziej obiektywne. Odpowiedzi na to pytanie były bardzo podobne do tych przedstawionych powyżej. Oprócz tego zapytano też, czy możliwość zostawienia dodatkowego wyróżnienia było dobrym pomysłem, na co 50 osób z 68 odpowiedziało twierdząco.
<!-- wykresy do któregoś z ostatnich 2 zdań tego akapitu? -->

Warsztaty z aplikacją Moodle wyróżniał dość istotny element, którego brakowało w autorskiej aplikacji. Podczas oceny studentom przedstawiane były konkretne kryteria bezpośrednio w aplikacji, sformułowane przez nauczyciela, według których studenci mieli ocenić projekty. W ankiecie zapytano, czy takie kryteria ułatwiają przeprowadzenie oceny.

In [54]:
qst_30 = df_form.filter(like='Czy w przypadku Warsztatów kryteria', axis=1)
counts_30 = qst_30.groupby(qst_30.columns[0]).size().reset_index(name="liczba")
counts_30.iloc[:,0] = counts_30.iloc[:,0].apply(lambda x : string_divider(x,interval=7, skip_short=True))
fig = px.pie(counts_30, values="liczba", names=qst_30.columns[0], title=string_divider(counts_30.columns[0]), width=700)
fig.update_layout(legend=dict(y=0.5), )

fig.update_layout(width=600, height=300, legend=dict(y=0.5))
fig.update_layout(margin=dict(l=10, r=10, t=60, b=10), title_y=0.92)

fig.show(config=config)

Ponad 80% oceniła takie kryteria jako przydatne. W odpowiedziach na pytania otwarte niektórzy twierdzili, że takie kryteria byłyby przydatne.
> "[...]Powinny też być jasno określone kryteria według których będzie przyznawana ocena jeszcze przed stworzeniem pracy. Dzięki temu praca zostanie stworzona pod konkretne wymagania."

> "Uważam, że ciekawą alternatywą dla rozdawania punktów byłoby przydzielanie od 1 do 5 gwiazdek dla każdej pracy w różnych kategoriach takich jak czytelność, dobór kolorów itd."

> "[...]do oceny powinny być udostępnione inne kryteria, na przykład takie które prowadzący przedmiot uwzględnił w notce (kolorystyka, przedstawienie, pomysł)[...]"






<!-- Na koniec przedstawione zostaną wybrane odpowiedzi na pytania otwarte sugerujące elementy do poprawy oraz wady zastosowanych rozwiązań.  -->
Na koniec postanowiono  wspomnieć o innych wybranych kwestiach poruszanych w odpowiedziach na pytania otwarte, w których studenci sugerowali, co można poprawić w aplikacji Zapunktuj oraz zostawiali inne uwagi i przemyślenia. Niektórzy pisali, że woleliby aby formułowanie pisemnej informacji zwrotnej było nieobowiązkowe. Kilka osób opisało problemy z błędnym wyświetlaniem niektórych informacji na stronie oraz problemem z przesłaniem formularza przez odświeżanie strony. Błędy te zostały wyeliminowane. Zwracano uwagę także na dostępność wyników oceniania. W aplikacji Zapunktuj były one dostępne na studentów, dopiero kiedy administrator grupy zakończył proces oceniania. W trakcie procesu wyniki były niedostępne, na co skarżyli się studenci ze względu na długie oczekiwanie na wyniki.  

Wszystkie wnioski z analizy zostały podsumowane w ostatnim rozdziale pracy magisterskiej.

---
---
---